In [10]:
%matplotlib inline
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

#load the data set + convert it to type numpy compatible with tensorflow
url = 'https://raw.githubusercontent.com/MyaraB/Deep-Learning-and-NLP/main/dataset_full.csv'
df1 = pd.read_csv(url)
df = pd.DataFrame(df1)
df = df.astype('float32')
df=df.to_numpy()

#shuffle data
np.random.shuffle(df)

#manually split the data into 4sets + 1 validation set
training_set1 = df[0:14080]
test_set1 = df[14080:17600]
training_set2 = df[17600:31680]
test_set2 = df[31680:35200]
training_set3 = df[35200:49280]
test_set3 = df[49280:52800]
training_set4 = df[52800:66880]
test_set4 = df[66880:70400]
validation_set= df[70400:] 

#manually choosing 14 relevant features from the 111 available to us in the dataset
#features used: qty_dot_url (1), qty_hyphen_url(2), qty_underline_url(3)
#,qty_slash_url(4),qty_equal_url(6), qty_at_url(7),qty_and_url(8), 
#qty_percent_url(17),qty_tld_url(18), length_url(19),qty_dot_domain(20),
#qty_hyphen_domain(21), qty_vowels_domain(37), domain_length(38),phishing(111)

training_set1_features = np.array(training_set1[:,[0,1,2,3,5,6,7,16,17,18,19,20,36,37]])
training_set1_labels = np.array(training_set1[:,-1])

test_set1_features = np.array(test_set1[:,[0,1,2,3,5,6,7,16,17,18,19,20,36,37]])
test_set1_labels = np.array(test_set1[:,-1])

features = 14
eps = 1e-12
learning_rates = 0.002

#shaping the data in-order for it to be usable with tensor
data_x = training_set1_features
test_x = test_set1_features
data_y = training_set1_labels.reshape((-1,1))
test_y = test_set1_labels.reshape((-1,1))

learning_rate = 0.002
features =14
hidden1_size=196
training_epochs = 14000

#pre proccesing..
x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 1])
W1 = tf.Variable(tf.truncated_normal([features, hidden1_size], stddev=0.1))
b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)
W2 = tf.Variable(tf.truncated_normal([hidden1_size, 1], stddev=0.1))
b2 = tf.Variable(tf.constant(0.1, shape=[1]))

y = tf.nn.sigmoid(tf.matmul(z1,W2)+b2)

loss=tf.reduce_mean(-(y_*tf.log(y + eps) + (1 - y_+eps)*tf.log(1 - y+eps)))
update = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

prediction = tf.round(tf.sigmoid(tf.matmul(z1,W2)+b2))
correct = tf.cast(tf.equal(prediction, y_), dtype=tf.float32) 
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(0,training_epochs+1):
  sess.run(update, feed_dict = {x:data_x, y_:data_y})
  if (epoch+1)%1000==0:
    err, _ = sess.run([loss, update], {x: data_x, y_:data_y})
    feeds_train = {x:data_x, y_:data_y}
    feeds_test = {x:test_x, y_:test_y}
    train_acc = sess.run(accuracy, feed_dict=feeds_train)
    test_acc = sess.run(accuracy, feed_dict=feeds_test)
    print ("epoch: %3d train accuracy: %.3f test accuracy: %.3f loss: %.3f" % (epoch+1,train_acc, test_acc, err))


epoch: 1000 train accuracy: 0.890 test accuracy: 0.893 loss: 0.312
epoch: 2000 train accuracy: 0.895 test accuracy: 0.897 loss: 0.291
epoch: 3000 train accuracy: 0.896 test accuracy: 0.897 loss: 0.280
epoch: 4000 train accuracy: 0.896 test accuracy: 0.897 loss: 0.273
epoch: 5000 train accuracy: 0.896 test accuracy: 0.898 loss: 0.267
epoch: 6000 train accuracy: 0.895 test accuracy: 0.898 loss: 0.262
epoch: 7000 train accuracy: 0.895 test accuracy: 0.898 loss: 0.258
epoch: 8000 train accuracy: 0.894 test accuracy: 0.899 loss: 0.255
epoch: 9000 train accuracy: 0.895 test accuracy: 0.900 loss: 0.252
epoch: 10000 train accuracy: 0.895 test accuracy: 0.900 loss: 0.249
epoch: 11000 train accuracy: 0.896 test accuracy: 0.901 loss: 0.247
epoch: 12000 train accuracy: 0.897 test accuracy: 0.901 loss: 0.245
epoch: 13000 train accuracy: 0.897 test accuracy: 0.902 loss: 0.243
epoch: 14000 train accuracy: 0.897 test accuracy: 0.901 loss: 0.241
